In [1]:
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout
from keras.applications import ResNet50
from keras.applications.vgg16 import preprocess_input

In [2]:
num_classes = 4
image_size = 224
batch_size_training = 10
batch_size_validation = 10

In [3]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,)

In [4]:
train_generator = data_generator.flow_from_directory(
    'New folder/train',
    target_size = (image_size, image_size),
    batch_size = batch_size_training,
    class_mode = 'categorical'
    )

validation_generator = data_generator.flow_from_directory(
    'New folder/validation',
    target_size = (image_size, image_size),
    batch_size = batch_size_training,
    class_mode = 'categorical'
    )

Found 315 images belonging to 4 classes.
Found 9 images belonging to 4 classes.


In [5]:
#adding VGG16 model
from xml.etree.ElementInclude import include

base_model = ResNet50(weights="imagenet", include_top=False, input_shape=[image_size, image_size, 3], pooling='max')
base_model.trainable = False

In [6]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(4096, activation='relu')
dense_layer_2 = layers.Dense(4096, activation='relu')
prediction_layer = layers.Dense(num_classes, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 4096)              8392704   
                                                                 
 dense_1 (Dense)             (None, 4096)              16781312  
                                                                 
 dense_2 (Dense)             (None, 4)                 16388     
                                                                 
Total params: 48,778,116
Trainable params: 25,190,404
Non-trainable params: 23,587,712
_________________________________________________________________


In [8]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

steps_per_epoch_train = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
number_epochs = 2




#model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

In [9]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

#early stopping is not used
fit_history = model.fit(train_generator, steps_per_epoch=steps_per_epoch_train, epochs=number_epochs,verbose=1, validation_data=validation_generator, validation_steps=steps_per_epoch_validation,)

Epoch 1/2
28/32 [=========================>....] - ETA: 3s - loss: 18.9649 - accuracy: 0.7927

c:\Users\ASUS\anaconda3\lib\site-packages\PIL\Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


32/32 [==============================] - 32s 877ms/step - loss: 16.5828 - accuracy: 0.8159 - val_loss: 2.7449 - val_accuracy: 0.7778
Epoch 2/2
32/32 [==============================] - 25s 790ms/step - loss: 0.2853 - accuracy: 0.9778 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


In [10]:
model.save('model/ResNet50_latter_training/resnet50_model_same_image.h5')

In [11]:
from keras.models import load_model
vgg16_saved = load_model('model/ResNet50_latter_training/resnet50_model_same_image.h5')

In [12]:
from keras.models import Model
basemodel1 = Model(inputs=vgg16_saved.input, outputs=vgg16_saved.get_layer('dense').output)

In [13]:
import numpy as np
from numpy.linalg import norm

In [15]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test4.jpg', target_size=(224, 224))
image2 = load_img('New folder/test/test3.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel1.predict(image1)
yhat2_base = basemodel1.predict(image2)

#cosine similarity
A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

1/1 [==============================] - 0s 118ms/step
Cosine Similarity: 0.95220345


In [16]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test4.jpg', target_size=(224, 224))
image2 = load_img('New folder/test/test.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel1.predict(image1)
yhat2_base = basemodel1.predict(image2)

#cosine similarity
A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

1/1 [==============================] - 0s 125ms/step
Cosine Similarity: 0.6813183


In [17]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test.png', target_size=(224, 224))
image2 = load_img('New folder/test/test3.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel1.predict(image1)
yhat2_base = basemodel1.predict(image2)

#cosine similarity
A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

1/1 [==============================] - 0s 115ms/step
Cosine Similarity: 0.813277


In [18]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test4.jpg', target_size=(224, 224))
image2 = load_img('New folder/test/test3.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel1.predict(image1)
yhat2_base = basemodel1.predict(image2)

#cosine similarity
A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

1/1 [==============================] - 0s 116ms/step
Cosine Similarity: 0.95220345


In [19]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test3.png', target_size=(224, 224))
image2 = load_img('New folder/test/test2.jpg', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel1.predict(image1)
yhat2_base = basemodel1.predict(image2)

#cosine similarity
A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

1/1 [==============================] - 0s 121ms/step
Cosine Similarity: 0.6869433
